# How do I make my pandas data frame smaller and faster?

In [1]:
import pandas as pd

In [2]:
drinks = pd.read_csv('drinks.csv')
drinks

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,South America
189,Vietnam,111,2,1,2.0,Asia
190,Yemen,6,0,0,0.1,Asia
191,Zambia,32,19,4,2.5,Africa


- Podemos ver el uso de memoria por columna con la funcion .info() ademas de que nos dice que tipo de datos se maneja en cada columna

In [3]:
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   continent                     193 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


- la memoria usada es solo un estimado que arroja debido a que los datos que son objetos pueden almacenar mas informacion de la que parece (esto por trartarse de strings convinados con numeros)
- podemos obligar a pandas para que nos de el estimado de la memoria estimada en la misma funcion .info() pero agregando un parametro;

In [4]:
drinks.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   continent                     193 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 30.5 KB


Como podemos notar ahora nos dice el verdadero tamaño de la memoria usada, que es de aprox 30.5 KB

In [5]:
drinks.memory_usage()

Index                            128
country                         1544
beer_servings                   1544
spirit_servings                 1544
wine_servings                   1544
total_litres_of_pure_alcohol    1544
continent                       1544
dtype: int64

- Para poder ver la memoria usada en KB reales podemos usar el mismo parametro en esta funcion;

In [6]:
drinks.memory_usage(deep=True)

Index                             128
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                       12332
dtype: int64

- Y para poder ver la suma de toda esta memoria podemos usar la funcion sum()

In [7]:
drinks.memory_usage(deep=True).sum()

31224

 #### Reducir los dtype que son object
 
 - Los datos que son de tipo object ocupan demasiada memoria 
 - lo que se propone a continuacion es tomar los tipo object y almacenarlo como integers
 - La primera duda que puede surgir es; ¿como un objeto que tiene mas forma de string puede hacerse integer?
 - Bueno, usando un metodo de pandas lo que haremos es almacenar las variables categorias en numeros seriados y finitos, empezando en este ejemplo por los continentes, ya que es una columna que tiene muy pocos uniquevalues

In [9]:
sorted(drinks.continent.unique())

['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

In [10]:
drinks.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: object

In [12]:
drinks['continent'] = drinks.continent.astype('category')

In [15]:
drinks.dtypes

country                           object
beer_servings                      int64
spirit_servings                    int64
wine_servings                      int64
total_litres_of_pure_alcohol     float64
continent                       category
dtype: object

- Podemos notar que ahora el tipo de objeto de continent es category

In [16]:
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   country                       193 non-null    object  
 1   beer_servings                 193 non-null    int64   
 2   spirit_servings               193 non-null    int64   
 3   wine_servings                 193 non-null    int64   
 4   total_litres_of_pure_alcohol  193 non-null    float64 
 5   continent                     193 non-null    category
dtypes: category(1), float64(1), int64(3), object(1)
memory usage: 8.1+ KB


- Y nos dice que en esta columna hay 6 catogorias:

In [17]:
drinks.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: category
Categories (6, object): [Africa, Asia, Europe, North America, Oceania, South America]

In [19]:
#Con esto podemos ver los codigos que fueron asignados a las catogorias para poder optimizar la memoria
drinks.continent.cat.codes.head()

0    1
1    2
2    0
3    2
4    0
dtype: int8

In [20]:
#Vamos a ver la memoria usada con estas modificaciones:
drinks.memory_usage(deep=True)

Index                             128
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         744
dtype: int64

#### Tipo categoria es optimizado solo si las catogorias son pocas

- Convertir a los continentes en tipo catogory funciono porque solo eran seis categorias
- Si hemos de intentar lo mismo en la columba de country notaremos que los uniquevalues son muchos mas de los que encontramos en continent:

In [23]:
sorted(drinks.country.unique())

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua & Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia-Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'DR Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hungary',
 'Iceland',


In [33]:
len(drinks.country.unique())

193

- Por lo que si hacemos el proceso anterior en la columna de country, usaremos mas memoria de lo que haciamos al principio

In [34]:
drinks['country'] = drinks.country.astype('category')

In [35]:
drinks.memory_usage(deep=True)

Index                             128
country                         18094
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         744
dtype: int64

Lo que ha pasado es que astype tuvo que crear 193 codigos diferentes para la columna country (1 codigo para cada pais)

In [36]:
drinks.country.cat.categories

Index(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua & Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria',
       ...
       'United Arab Emirates', 'United Kingdom', 'Uruguay', 'Uzbekistan',
       'Vanuatu', 'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', length=193)

#### Es conveniente usar astype = categories solo cuando los valores unicos de la serie son muy pocos

##### Bonus:

In [41]:
df = pd.DataFrame({'ID':[100, 101, 102, 103], 'quality':['superb', 'very good', 'good', 'excelent']})
df

,ID,quality
0,100,superb
1,101,very good
2,102,good
3,103,excelent


In [42]:
df.sort_values('quality')

,ID,quality
3,103,excelent
2,102,good
0,100,superb
1,101,very good


- Supongamos que para nosotros la quality tiene un orden logico, es decir que algunas quality's son mas relevantes que otras
- Y lo que ahora pretendemos es que nuestro df sea ordenado por medio de quality

In [47]:
#Esta era la vieja forma de armar la jerarquia de las categorias dentro de la asignacion
#Actualmente no sirve, btw
df['quality'] = df.quality.astype('category', categories = ['good', 'very good', 'superb', 'excelent'], ordered = True)
df.quality

TypeError: astype() got an unexpected keyword argument 'categories'

- La nueva forma de armar la jerarquizacion de las categorias es declararla por separado:

In [58]:
from pandas.api.types import CategoricalDtype
quality_car = CategoricalDtype(['good', 'very good', 'superb', 'excelent'], ordered = True)

In [59]:
df['quality'] = df.quality.astype(quality_car)
df

,ID,quality
0,100,superb
1,101,very good
2,102,good
3,103,excelent


In [57]:
df.quality

0       superb
1    very good
2         good
3     excelent
Name: quality, dtype: category
Categories (4, object): [good < very good < superb < excelent]

- Ahora podemos ver que las categorias tienen gearquia y si hacemos sort al df con quality como argumento tomara en cuenta esta jerarquizacion y no el orden alfabetico que tenia antes

In [60]:
df.sort_values('quality')

,ID,quality
2,102,good
1,101,very good
0,100,superb
3,103,excelent


- Esto es util porque nos permite hacer ahora un filtro en base a caracteristicas especificas que hemos definido:

In [61]:
df.loc[df.quality > 'good',:]

,ID,quality
0,100,superb
1,101,very good
3,103,excelent
